Importing the Image Data Generator Libraries 

In [1]:
import tensorflow as tf 
import keras 
from keras.preprocessing.image import ImageDataGenerator

Canfiguring The Image Data Generator 

In [73]:
#training images 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

#val images 
val_datagen = ImageDataGenerator(rescale=1./255)

Applying the image generator to the body 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [77]:
body_train_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Dataset/Car damage/body/training',
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

Found 979 images belonging to 3 classes.


In [62]:
body_val_set = val_datagen.flow_from_directory(
        '/content/drive/MyDrive/Dataset/Car damage/body/validation',
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

Found 171 images belonging to 3 classes.


Applying the image generator to the level 

In [75]:
level_train_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Dataset/Car damage/level/training',
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

Found 979 images belonging to 3 classes.


In [86]:
level_val_set = val_datagen.flow_from_directory(
        '/content/drive/MyDrive/Dataset/Car damage/level/validation',
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

Found 175 images belonging to 3 classes.


In [80]:
from keras.layers import Dense, Flatten, Input 
from keras.models import Model 
from keras.applications.vgg16 import VGG16, preprocess_input

In [87]:
vgg = VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in vgg.layers:
  layer.trainable = False
x = Flatten()(vgg.output)
prediction = Dense(3,activation='softmax')(x)

In [88]:
model = Model(inputs=vgg.input,outputs = prediction)
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [89]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [90]:
import sys 
model.fit(body_train_set,validation_data=level_val_set,epochs=25,steps_per_epoch=979//10,validation_steps=175//10)

Epoch 1/25
97/97 [==============================] - 30s 305ms/step - loss: 1.2089 - accuracy: 0.5511 - val_loss: 2.7505 - val_accuracy: 0.2353
Epoch 2/25
97/97 [==============================] - 16s 166ms/step - loss: 0.6349 - accuracy: 0.7358 - val_loss: 2.7987 - val_accuracy: 0.2647
Epoch 3/25
97/97 [==============================] - 16s 162ms/step - loss: 0.4991 - accuracy: 0.8163 - val_loss: 3.0798 - val_accuracy: 0.2765
Epoch 4/25
97/97 [==============================] - 15s 155ms/step - loss: 0.4013 - accuracy: 0.8473 - val_loss: 3.7942 - val_accuracy: 0.2588
Epoch 5/25
97/97 [==============================] - 17s 170ms/step - loss: 0.2752 - accuracy: 0.9020 - val_loss: 4.0837 - val_accuracy: 0.2294
Epoch 6/25
97/97 [==============================] - 15s 158ms/step - loss: 0.2741 - accuracy: 0.8958 - val_loss: 4.5609 - val_accuracy: 0.2765
Epoch 7/25
97/97 [==============================] - 15s 156ms/step - loss: 0.2568 - accuracy: 0.8989 - val_loss: 3.8147 - val_accuracy: 0.3176

In [69]:
model.save('body.h5')

In [91]:
import sys 
model.fit(level_train_set,validation_data=body_val_set,epochs=25,steps_per_epoch=979//10,validation_steps=175//10)

Epoch 1/25
97/97 [==============================] - 562s 6s/step - loss: 4.3518 - accuracy: 0.4861 - val_loss: 9.0484 - val_accuracy: 0.3000
Epoch 2/25
97/97 [==============================] - 15s 151ms/step - loss: 2.4040 - accuracy: 0.6543 - val_loss: 10.1531 - val_accuracy: 0.2882
Epoch 3/25
97/97 [==============================] - 15s 152ms/step - loss: 1.5395 - accuracy: 0.7265 - val_loss: 10.5255 - val_accuracy: 0.2353
Epoch 4/25
97/97 [==============================] - 16s 165ms/step - loss: 1.0665 - accuracy: 0.7771 - val_loss: 10.5748 - val_accuracy: 0.2412
Epoch 5/25
97/97 [==============================] - 15s 155ms/step - loss: 0.6536 - accuracy: 0.8442 - val_loss: 9.3509 - val_accuracy: 0.2471
Epoch 6/25
97/97 [==============================] - 15s 153ms/step - loss: 0.3836 - accuracy: 0.8958 - val_loss: 11.8252 - val_accuracy: 0.2412
Epoch 7/25
97/97 [==============================] - 16s 165ms/step - loss: 0.3693 - accuracy: 0.8762 - val_loss: 9.6993 - val_accuracy: 0.26

In [92]:
model.save('level.h5')